# AIM

ニューラルネットワークでアイリスデータ分類 ver. softmax with Chainer

In [8]:
import time
import numpy as np
import pandas as pd
from sklearn import datasets
import chainer
from chainer import Chain, optimizers, training
from chainer.training import extensions
import chainer.functions as F
import chainer.links as L

In [9]:
from pkg_resources import get_distribution
import platform
print("python", platform.python_version())
print("")
libs = ["numpy", "pandas", "scikit-learn", "chainer"]
for lib in libs:
    version = get_distribution(lib).version
    print(lib, version)

python 3.5.2

numpy 1.13.1
pandas 0.20.3
scikit-learn 0.18.2
chainer 2.0.2


In [10]:
# モデルクラス定義

class NN(Chain):
    def __init__(self, in_size, hidden_size, out_size):
        # クラスの初期化
        # :param in_size: 入力層のサイズ
        # :param hidden_size: 隠れ層のサイズ
        # :param out_size: 出力層のサイズ
        super(NN, self).__init__(
            xh = L.Linear(in_size, hidden_size),
            hh = L.Linear(hidden_size, hidden_size),
            hy = L.Linear(hidden_size, out_size)
        )
 
    def __call__(self, x):
        # 順伝播の計算を行う関数
        # :param x: 入力値
        h = F.sigmoid(self.xh(x))
        h = F.sigmoid(self.hh(h))
        y = self.hy(h)
        return y

In [12]:
# 学習

EPOCH_NUM = 100
HIDDEN_SIZE = 20
BATCH_SIZE = 20

# データ
N = 100
in_size = 4
out_size = 3
iris = datasets.load_iris()
data = pd.DataFrame(data= np.c_[iris["data"], iris["target"]], columns= iris["feature_names"] + ["target"])
data = np.array(data.values)
dataset = []
for d in data:
    x = d[0:4]
    y = d[4]
    dataset.append((np.array(x, dtype="float32"), np.array(y, dtype="int32")))
N = len(dataset)

# モデルの定義
model = L.Classifier(NN(in_size=in_size, hidden_size=HIDDEN_SIZE, out_size=out_size))
optimizer = optimizers.Adam()
optimizer.setup(model)

# 学習開始
print("Train")
train, test = chainer.datasets.split_dataset_random(dataset, N-50) # 100件を学習用、50件をテスト用
train_iter = chainer.iterators.SerialIterator(train, BATCH_SIZE)
test_iter = chainer.iterators.SerialIterator(test, BATCH_SIZE, repeat=False, shuffle=False)
updater = training.StandardUpdater(train_iter, optimizer, device=-1)
trainer = training.Trainer(updater, (EPOCH_NUM, "epoch"), out="result")
trainer.extend(extensions.Evaluator(test_iter, model, device=-1))
trainer.extend(extensions.LogReport(trigger=(10, "epoch"))) # 10エポックごとにログ出力
trainer.extend(extensions.PrintReport( ["epoch", "main/loss", "validation/main/loss", "main/accuracy", "validation/main/accuracy", "elapsed_time"])) # エポック、学習損失、テスト損失、学習正解率、テスト正解率、経過時間
#trainer.extend(extensions.ProgressBar()) # プログレスバー出力
trainer.run()

Train
epoch       main/loss   validation/main/loss  main/accuracy  validation/main/accuracy  elapsed_time
10          1.11115     1.08219               0.365          0.39                      0.11087       
20          1.03875     1.07909               0.408          0.246667                  0.237742      
30          0.965692    1.02292               0.664          0.568333                  0.342483      
40          0.855172    0.926833              0.7            0.6                       0.45431       
50          0.720153    0.806095              0.701          0.641667                  0.567012      
60          0.598405    0.696131              0.766          0.698333                  0.68277       
70          0.511003    0.613085              0.878          0.805                     0.795234      
80          0.444817    0.544701              0.958          0.933333                  0.901574      
90          0.394639    0.488703              0.97           0.966667         

In [22]:
# 予測

print("Predict")
print("x\ty\tpredict")
idx = np.random.choice(N, 10)
for i in idx:
    x = dataset[i][0]
    y_ = np.argmax(model.predictor(x=x.reshape(1,len(x))).data)
    y = dataset[i][1]
    print(x, "\t", y, "\t", y_)

Predict
x	y	predict
[ 6.19999981  2.79999995  4.80000019  1.79999995] 	 2 	 2
[ 5.30000019  3.70000005  1.5         0.2       ] 	 0 	 0
[ 6.69999981  3.0999999   4.69999981  1.5       ] 	 1 	 1
[ 4.9000001   2.4000001   3.29999995  1.        ] 	 1 	 1
[ 5.69999981  3.79999995  1.70000005  0.30000001] 	 0 	 0
[ 5.69999981  3.          4.19999981  1.20000005] 	 1 	 1
[ 4.9000001  3.0999999  1.5        0.1      ] 	 0 	 0
[ 5.5         2.29999995  4.          1.29999995] 	 1 	 1
[ 6.9000001  3.0999999  4.9000001  1.5      ] 	 1 	 1
[ 6.5         2.79999995  4.5999999   1.5       ] 	 1 	 1
